In [1]:
!pip install pyngrok
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.6/793.6 kB 8.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
# Step 1: Install specific version
!pip install face_recognition==1.2.3

# Step 2: Restart the runtime (Uncomment the following line to run it)
#import os
#os._exit(00)

# Step 3: Verify installation (Run this in a new cell after restarting the runtime)
import face_recognition
print(face_recognition.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=99703c396c45c5434424b7caedf1a13d96c1d2f709a076be44a48309b8c1f03e
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models
1.2.3


In [3]:
# app.py
from flask import Flask, render_template, Response
import cv2
from pyngrok import ngrok
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np

app = Flask(__name__)

def get_video_stream():
    js = Javascript('''
        async function getVideo() {
            const div = document.createElement('div');
            const video = document.createElement('video');
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            document.body.appendChild(canvas);
            const context = canvas.getContext('2d');

            window.addEventListener('beforeunload', () => {
                stream.getTracks().forEach(track => track.stop());
            });

            return new Promise((resolve) => {
                const interval = setInterval(() => {
                    context.drawImage(video, 0, 0, canvas.width, canvas.height);
                    const dataURL = canvas.toDataURL('image/jpeg', 0.8);
                    clearInterval(interval);
                    resolve(dataURL);
                }, 100);  // Capture every 100 milliseconds
            });
        }
        getVideo();
    ''')
    display(js)
    data = eval_js("getVideo()")
    binary = b64decode(data.split(',')[1])
    np_data = np.frombuffer(binary, np.uint8)
    img = cv2.imdecode(np_data, cv2.IMREAD_COLOR)
    return img





In [4]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/Graduation Project /Deployment /Final_Whole_Website

/content/drive/.shortcut-targets-by-id/1tCxZF_4LbuiGQaCNrczmpjBOfcMC1Rgk/Graduation Project /Deployment /Final_Whole_Website


In [6]:
from pyngrok import ngrok
from flask import Flask,render_template,Response, request, send_file
import cv2
from YOLO_Video import video_detection
from simple_facerec import SimpleFacerec
from ultralytics import YOLO
import math
from werkzeug.utils import secure_filename
from keras.models import load_model
from collections import deque
import numpy as np
import os


In [7]:
from flask import Flask, render_template, Response, request, jsonify
import cv2
from pyngrok import ngrok
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import tensorflow as tf
from collections import deque
from werkzeug.utils import secure_filename
import os

In [8]:
# Streaming
def generate_frames():
    while True:
        frame = get_video_stream()
        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

#---------------------------------------------------------------------------------------------------
#Fire Detection
CLASSES_LIST_FIRE = ["Fire", "Smoke"]
def detect_fire_in_frame(frame):
    # Your fire detection model loading code here
    model = YOLO("best (3).pt")
    results = model(frame, stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 3)
            conf = np.ceil((box.conf[0].cpu() * 100)) / 100  # Move tensor to CPU
            cls = int(box.cls[0])
            class_name = CLASSES_LIST_FIRE[cls]
            label = f'{class_name} {conf}'
            t_size = cv2.getTextSize(label, 0, fontScale=1, thickness=2)[0]
            c2 = x1 + t_size[0], y1 - t_size[1] - 3
            cv2.rectangle(frame, (x1, y1), c2, [255, 0, 255], -1, cv2.LINE_AA)
            cv2.putText(frame, label, (x1, y1 - 2), 0, 1, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)
    return frame

def generate_fire_stream():
    while True:
        frame = get_video_stream()
        frame = detect_fire_in_frame(frame)
        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')


def detect_fire(video_path):
    model = YOLO("best (3).pt")
    cap = cv2.VideoCapture(video_path)
    output_path = os.path.join(app.config['UPLOAD_FOLDER'], 'processed_video.mp4')

    original_video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    video_writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'),
                                   cap.get(cv2.CAP_PROP_FPS), (original_video_width, original_video_height))

    frames_queue = deque(maxlen=SEQUENCE_LENGTH)
    key_frame_saved = False

    while cap.isOpened():
        ok, frame = cap.read()
        if not ok:
            break

        resized_frame = cv2.resize(frame, (64, 64))
        normalized_frame = resized_frame / 255
        frames_queue.append(normalized_frame)

        if len(frames_queue) == SEQUENCE_LENGTH:
            results = model(frame, stream=True)
            for r in results:
                boxes = r.boxes
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 3)
                    conf = np.ceil((box.conf[0].cpu() * 100)) / 100  # Move tensor to CPU
                    cls = int(box.cls[0])
                    class_name = CLASSES_LIST_FIRE[cls]
                    label = f'{class_name} {conf}'
                    t_size = cv2.getTextSize(label, 0, fontScale=1, thickness=2)[0]
                    c2 = x1 + t_size[0], y1 - t_size[1] - 3
                    cv2.rectangle(frame, (x1, y1), c2, [255, 0, 255], -1, cv2.LINE_AA)
                    cv2.putText(frame, label, (x1, y1 - 2), 0, 1, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)

            # Save this frame as a key frame if it hasn't been saved before
            if not key_frame_saved:
                key_frame_path = os.path.join(app.config['UPLOAD_FOLDER'], 'key_frame.jpg')
                cv2.imwrite(key_frame_path, frame)
                key_frame_saved = True

        video_writer.write(frame)

    cap.release()
    video_writer.release()
    return output_path
#-------------------------------------------------------------------------------------------------------------------
#Gun Detection
CLASSES_LIST_GUN = ["Pistol","Grenade" , "Gun" , "Knife"]
SEQUENCE_LENGTH = 16
def detect_gun(video_path):
    model = YOLO("best.pt")
    cap = cv2.VideoCapture(video_path)
    output_path = os.path.join(app.config['UPLOAD_FOLDER'], 'processed_video.mp4')

    original_video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    video_writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'),
                                   cap.get(cv2.CAP_PROP_FPS), (original_video_width, original_video_height))

    frames_queue = deque(maxlen=SEQUENCE_LENGTH)
    key_frame_saved = False

    while cap.isOpened():
        ok, frame = cap.read()
        if not ok:
            break

        resized_frame = cv2.resize(frame, (64, 64))
        normalized_frame = resized_frame / 255
        frames_queue.append(normalized_frame)

        if len(frames_queue) == SEQUENCE_LENGTH:
            results = model(frame, stream=True)
            for r in results:
                boxes = r.boxes
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 3)
                    conf = np.ceil((box.conf[0].cpu() * 100)) / 100  # Move tensor to CPU
                    cls = int(box.cls[0])
                    class_name = CLASSES_LIST_GUN[cls]
                    label = f'{class_name} {conf}'
                    t_size = cv2.getTextSize(label, 0, fontScale=1, thickness=2)[0]
                    c2 = x1 + t_size[0], y1 - t_size[1] - 3
                    cv2.rectangle(frame, (x1, y1), c2, [255, 0, 255], -1, cv2.LINE_AA)
                    cv2.putText(frame, label, (x1, y1 - 2), 0, 1, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)

            # Save this frame as a key frame if it hasn't been saved before
            if not key_frame_saved:
                key_frame_path = os.path.join(app.config['UPLOAD_FOLDER'], 'key_frame.jpg')
                cv2.imwrite(key_frame_path, frame)
                key_frame_saved = True

        video_writer.write(frame)

    cap.release()
    video_writer.release()
    return output_path

def detect_gun_in_frame(frame):
    # Placeholder function for gun detection
    # Implement your gun detection model here
    model = YOLO("best.pt")
    results = model(frame, stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 3)
            conf = np.ceil((box.conf[0].cpu() * 100)) / 100  # Move tensor to CPU
            cls = int(box.cls[0])
            class_name = CLASSES_LIST_GUN[cls]
            label = f'{class_name} {conf}'
            t_size = cv2.getTextSize(label, 0, fontScale=1, thickness=2)[0]
            c2 = x1 + t_size[0], y1 - t_size[1] - 3
            cv2.rectangle(frame, (x1, y1), c2, [255, 0, 255], -1, cv2.LINE_AA)
            cv2.putText(frame, label, (x1, y1 - 2), 0, 1, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)
    return frame

def generate_gun_stream():
    while True:
        frame = get_video_stream()
        frame = detect_gun_in_frame(frame)
        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')


#----------------------------------------------------------------------------------------------------------------------
#Violence
CLASSES_LIST = ["NonViolence","Violence"]
SEQUENCE_LENGTH=16
latest_prediction = "No prediction yet"  # Global variable to store the latest prediction
IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64
model = tf.keras.models.load_model('best_weights.h5')

def process_video(video_path, model_path):
    model = load_model(model_path)
    vs = cv2.VideoCapture(video_path)
    output_path = os.path.join(app.config['UPLOAD_FOLDER'], 'processed_video.avi')

    original_video_width = int(vs.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(vs.get(cv2.CAP_PROP_FRAME_HEIGHT))

    video_writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc('m', 'p', '4', 'v'),
                                   vs.get(cv2.CAP_PROP_FPS), (original_video_width, original_video_height))

    frames_queue = deque(maxlen=SEQUENCE_LENGTH)
    predicted_class_name = ''
    key_frame_saved = False

    while vs.isOpened():
        ok, frame = vs.read()
        if not ok:
            break

        resized_frame = cv2.resize(frame, (64, 64))
        normalized_frame = resized_frame / 255
        frames_queue.append(normalized_frame)

        if len(frames_queue) == SEQUENCE_LENGTH:
            predicted_labels_probabilities = model.predict(np.expand_dims(frames_queue, axis=0))[0]
            predicted_label = np.argmax(predicted_labels_probabilities)
            predicted_class_name = CLASSES_LIST[predicted_label]
            max_probability = np.max(predicted_labels_probabilities)
            percentage = max_probability * 100

            # Set text color based on the prediction
            if predicted_class_name == "Violence":
                text_color = (0, 0, 255)  # Red color for Violence class
            else:
                text_color = (0, 255, 0)  # Green color for NonViolence class

            text = f'{predicted_class_name}: {percentage:.2f}%'
            cv2.putText(frame, text, (2, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, text_color, 4)

            # Save this frame as a key frame if the condition meets and it hasn't been saved before
            if not key_frame_saved:
                key_frame_path = os.path.join(app.config['UPLOAD_FOLDER'], 'key_frame.jpg')
                cv2.imwrite(key_frame_path, frame)
                key_frame_saved = True

        video_writer.write(frame)

    vs.release()
    video_writer.release()
    return output_path

def preprocess_image(image_data, frame_width, frame_height):
    resized_frame = cv2.resize(image_data, (frame_width, frame_height))
    normalized_frame = resized_frame / 255.0
    return normalized_frame

def generate_violence_stream():
    global latest_prediction
    frames_buffer = deque(maxlen=SEQUENCE_LENGTH)
    while True:
        frame = get_video_stream()
        if frame is None:
            print("Frame capture failed")
            continue
        frame_processed = preprocess_image(frame, IMAGE_WIDTH, IMAGE_HEIGHT)
        if frame_processed is None:
            print("Frame preprocessing failed")
            continue
        frames_buffer.append(frame_processed)

        if len(frames_buffer) == SEQUENCE_LENGTH:
            frames = np.array(frames_buffer)
            frames = np.expand_dims(frames, axis=0)  # Add batch dimension

            # Ensure the frames have the correct shape
            print(f"Frames shape: {frames.shape}")  # Debug print to check the shape

            prediction = model.predict(frames)
            predicted_class = np.argmax(prediction, axis=1)[0]
            label = CLASSES_LIST[predicted_class]
            latest_prediction = label  # Update the global prediction

            # Draw the prediction label on the frame
            cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        ret, frame_buffer = cv2.imencode('.jpg', frame)
        frame = frame_buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')




In [9]:
!pip install flask_sqlalchemy
!pip install flask_login
!pip install bcrypt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 5.7 MB/s eta 0:00:00


In [10]:
from flask import Flask, render_template, redirect, request,session, Response,url_for
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, login_user, login_required
from sqlalchemy.exc import IntegrityError
from bcrypt import hashpw, gensalt,checkpw

In [11]:

app=Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///../database/users.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS']  = False
app.config['SECRET_KEY']='a92cf3597ef6066d2aacb1c37b6bbf4dd879e9e08f578fe9b4c619b3224d9ed7'

camera=cv2.VideoCapture(1)
app.config['UPLOAD_FOLDER'] = 'static/videos'



db = SQLAlchemy(app)
app.app_context().push()

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(20), unique=True, nullable=False)
    email = db.Column(db.String(100), unique=True, nullable=False)
    password = db.Column(db.String(120), nullable=False)

    def __init__(self,username,email,password):
        self.username = username
        self.email = email
        self.password = password

login_manager = LoginManager()
login_manager.init_app(app)


@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/login", methods=['GET','POST'])
def login():
    if request.method == "POST":
        username = request.form['username']
        password = request.form['password']

        user = User.query.filter_by(username=username).first()

        if user:
            if checkpw(password.encode('utf-8'), user.password):
                session['username'] = user.username
                session['password'] = user.password
                # return render_template("fire.html")
                return redirect('/fire')

            else:
                error = "Invalid password"
                return render_template("Login.html", error=error)
        else:
            error = "Invalid username"
            return render_template("Login.html", error=error)

    return render_template("Login.html")

@app.route("/register", methods=['GET','POST'])
def register():
    if request.method == "POST":
        username = request.form['username']
        email = request.form['email']
        password = request.form['password']
        confirm_password = request.form['confirm_password']

        # Check if passwords match
        if password != confirm_password:
            error = "Passwords do not match"
            return render_template("Register.html", error=error)

        # Hash the password before storing
        hashed_password = hashpw(password.encode('utf-8'),gensalt() )
        try:
            new_user = User(username=username,email=email,password=hashed_password)
            db.session.add(new_user)
            db.session.commit()
            return redirect("/login")

        except IntegrityError as e:
            db.session.rollback()  # Rollback the transaction
            error = None
            if 'UNIQUE constraint failed: user.username' in str(e):
                error = "Username already exists"
            elif 'UNIQUE constraint failed: user.email' in str(e):
                error = "Email already exists"
            else:
                error = "An error occurred, please try again."
            return render_template("Register.html", error=error)
    else:
        return render_template("Register.html")

In [12]:
#-----------------------------------------------------------------------------------
# Fire Page
@app.route('/fire')
def fire():
    if 'username' in session:
        return render_template("fire.html",username=session)
    else:
        return redirect('/login')

ALLOWED_EXTENSIONS = ['mp4']

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/upload', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        if 'video' not in request.files:
            return 'No video file found'

        f = request.files['video']

        if f.filename == '':
            return 'No video file selected'

        filename = secure_filename(f.filename)
        upload_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        f.save(upload_path)
        processed_video_path = detect_fire(upload_path)

        key_frame_path = os.path.join(app.config['UPLOAD_FOLDER'], 'key_frame.jpg')
        return render_template('Results_Fire.html', video_file=os.path.basename(processed_video_path), key_frame=os.path.basename(key_frame_path))
    return render_template('fire.html')

@app.route('/download/<filename>')
def download(filename):
    return send_file(os.path.join(app.config['UPLOAD_FOLDER'], filename), as_attachment=True)

@app.route('/video_fire_stream')
def video_fire_stream():
    return Response(generate_fire_stream(), mimetype='multipart/x-mixed-replace; boundary=frame')


#--------------------------------------------------------------------------------------------------
# Gun Page
@app.route('/gun')
def gun():
    return render_template('gun.html')

@app.route('/upload_gun', methods=['GET', 'POST'])
def upload_gun():
    if request.method == 'POST':
        if 'video' not in request.files:
            return 'No video file found'

        f = request.files['video']

        if f.filename == '':
            return 'No video file selected'

        filename = secure_filename(f.filename)
        upload_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        f.save(upload_path)
        processed_video_path = detect_gun(upload_path)

        key_frame_path = os.path.join(app.config['UPLOAD_FOLDER'], 'key_frame.jpg')
        return render_template('Results_gun.html', video_file=os.path.basename(processed_video_path), key_frame=os.path.basename(key_frame_path))
    return render_template('gun.html')

@app.route('/download/<filename>')
def download_gun(filename):
    return send_file(os.path.join(app.config['UPLOAD_FOLDER'], filename), as_attachment=True)

@app.route('/video_gun_stream')
def video_gun_stream():
    return Response(generate_gun_stream(), mimetype='multipart/x-mixed-replace; boundary=frame')


#---------------------------------------------------------------------------------------------------------------------
# Recognition Page
app.config['UPLOAD_FOLDER'] = 'static/videos'
ALLOWED_EXTENSIONS = ['mp4']

# Load face recognition model and encodings
sfr = SimpleFacerec()
sfr.load_encoding_images("images/")  # Update the path accordingly
def detect_known_faces_in_frame(frame):
    face_locations, face_names = sfr.detect_known_faces(frame)
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 0, 255), 1)
    return frame

def generate_recog_stream():
    while True:
        frame = get_video_stream()
        frame = detect_known_faces_in_frame(frame)
        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')



@app.route('/recog')
def recog():
    return render_template('recog.html')


def detect_faces(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    out = cv2.VideoWriter(os.path.join(app.config['UPLOAD_FOLDER'], 'output.mp4'),
                          cv2.VideoWriter_fourcc(*'mp4v'), 10, (frame_width, frame_height))
    key_frame_path = os.path.join(app.config['UPLOAD_FOLDER'], 'key_frame.jpg')
    key_frame_saved = False

    while True:
        success, img = cap.read()
        if not success:
            break

        face_locations, face_names = sfr.detect_known_faces(img)
        for face_loc, name in zip(face_locations, face_names):
            y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]
            cv2.putText(img, name, (x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 200), 2)
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 200), 4)

        if face_names and not key_frame_saved:
            cv2.imwrite(key_frame_path, img)
            key_frame_saved = True

        out.write(img)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    if not key_frame_saved:
        cap = cv2.VideoCapture(video_path)
        success, img = cap.read()
        if success:
            cv2.imwrite(key_frame_path, img)
        cap.release()

    return key_frame_path


@app.route('/upload_Recog', methods=['GET', 'POST'])
def upload_Recog():
    if 'video' not in request.files:
        return 'No video file found'

    video = request.files['video']
    if video.filename == '':
        return 'No video file selected'

    if video and allowed_file(video.filename):
        video_path = os.path.join(app.config['UPLOAD_FOLDER'], video.filename)
        video.save(video_path)

        key_frame_path = detect_faces(video_path)
        return render_template('Results_Recog.html', video_name='output.mp4', key_frame=os.path.basename(key_frame_path))

    return 'Invalid file type'


@app.route('/download/<filename>')
def download_Recog(filename):
    detected_video_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    return send_file(detected_video_path, as_attachment=True)

@app.route('/video_recog_stream')
def video_recog_stream():
    return Response(generate_recog_stream(), mimetype='multipart/x-mixed-replace; boundary=frame')


#----------------------------------------------------------------------------------------------------------------------
# Violence Page
app.config['UPLOAD_FOLDER'] = 'static/videos'
@app.route('/violence')
def violence():
    return render_template('violence.html')
@app.route('/uploader', methods=['GET', 'POST'])
def uploader():
    if request.method == 'POST':
        f = request.files['file']
        filename = secure_filename(f.filename)
        upload_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        f.save(upload_path)
        processed_video_path = process_video(upload_path, 'best_weights.h5')

        # Corrected line: Use 'UPLOAD_FOLDER' instead of 'UPLOAD_API'
        key_frame_path = os.path.join(app.config['UPLOAD_FOLDER'], 'key_frame.jpg')
        return render_template('Results_Violence.html', video_file=os.path.basename(processed_video_path), key_frame=os.path.basename(key_frame_path))

@app.route('/video_violence_stream')
def video_violence_stream():
    return Response(generate_violence_stream(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/violence_prediction', methods=['GET'])
def violence_prediction():
    global latest_prediction
    return jsonify({"prediction": latest_prediction})


@app.route('/download/<filename>')
def download_file(filename):
    return send_file(os.path.join(app.config['UPLOAD_FOLDER'], filename), as_attachment=True)


3 encoding images found.
Encoding images loaded


In [15]:

if __name__ == "__main__":
   ngrok.set_auth_token("2iBxvSm3FTTtG79Qs9qVVBOBQgP_3nMCR13xT6uf9jZraW7sQ")
   ngrok_tunnel = ngrok.connect(5000)
   print('Public URL:', ngrok_tunnel.public_url)
   app.run()

Public URL: https://9462-34-125-12-103.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/Jun/2024 13:04:18] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jun/2024 13:04:18] "GET /static/Lib/animate/animate.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jun/2024 13:04:18] "GET /static/Lib/owlcarousel/assets/owl.carousel.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jun/2024 13:04:18] "GET /static/img/face.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jun/2024 13:04:18] "GET /static/css/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jun/2024 13:04:18] "GET /static/img/icon/face-icon.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jun/2024 13:04:18] "GET /static/css/bootstrap.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jun/2024 13:04:18] "GET /static/img/camera.jpg HTTP/1.1" 200 -
INF